In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import CollisionFilterDeclaration, GeometrySet
import meshcat

# Build plant

In [17]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

gripper_geometries = GeometrySet()
for name in ["body", "left_finger", "right_finger"]:
    body = plant.GetBodyByName(name)
    print(body)
    frame_id = plant.GetBodyFrameIdOrThrow(body.index())
    gripper_geometries.Add(frame_id)

scene_graph.collision_filter_manager().Apply(CollisionFilterDeclaration().ExcludeWithin(gripper_geometries))



<RigidBody_[float] name='body' index=11 model_instance=3>
<RigidBody_[float] name='left_finger' index=12 model_instance=3>
<RigidBody_[float] name='right_finger' index=13 model_instance=3>


In [18]:
print(ret)

None


# Setup meshcat visualization

In [19]:
do_viz = False
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)
diagram = visualizer.diagram

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6002...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connected to meshcat-server.


# Run SNOPT IRIS in rational space

In [20]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.configuration_space_margin = 1e-5
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False
# uncomment to test if non-default is working
# currently throwing std::bad_alloc if we try to set q_star
iris_options.q_star = 0.5 * np.zeros(3)
rational_forward_kinematics = RationalForwardKinematics(plant)


#seed points specified in q space
seed_points_q = np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
#                         [0.3, -0.8, 0.5],     # above shelf
#                         [0.25, -1.6, -0.25],  # in shelf 1
#                         [0.07, -1.8, -0.2],   # leaving shelf 1
#                         [-0.1, -2, -0.3]      # out of shelf 1
                        ]) 
seed_points_t = np.array([rational_forward_kinematics.ComputeTValue(s, iris_options.q_star) for s in seed_points_q])


In [21]:
regions = []
context = diagram.CreateDefaultContext()
for i, s in enumerate(seed_points_q):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if i > 0:
        starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points_q)}')

Completed region: 1/2
Completed region: 2/2


In [22]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions)
    visualizer.plot_seedpoints(seed_points_t)
    visualizer.visualize_collision_constraint(N = 50)

# Begin Certification

In [23]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

In [24]:
editted_regions = []
filtered_collision_pairs = set()
bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 2
bilinear_alternation_option.lagrangian_backoff_scale = 0.01
bilinear_alternation_option.polytope_backoff_scale = 0.05
solver_options = mp.SolverOptions()
for r in regions: 
    C_final, d_final, P_final, q_final = \
        cspace_free_region.CspacePolytopeBilinearAlternation(
            iris_options.q_star, filtered_collision_pairs, r.A(), r.b(),
            bilinear_alternation_option, solver_options)
    editted_regions.append(HPolyhedron(C_final, d_final))

RuntimeError: GenerateMonomialBasisWithOrderUpToOneHelper: Shouldn't have an empty input t_angles.

In [ ]:
kin = cspace_free_region.rational_forward_kinematics

In [ ]:
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(diagram.CreateDefaultContext()))
inspector = query.inspector()
ids = plant.GetBodyIndices()
print(ids)

In [ ]:
from pydrake.all import GeometryId
name = inspector.GetName(GeometryId(8))